In [30]:
import pyaudio
import wave
import sys
import requests
import time

import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np

from pydub import AudioSegment

%matplotlib inline

plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
def record(filename, time_s, rate=44100):

    chunk = 1024
    audio = pyaudio.PyAudio()

    # start recording
    stream = audio.open(format=pyaudio.paInt16,
                        channels=2,
                        rate=rate,
                        input=True,
                        frames_per_buffer=chunk)

    print("recording...")
    frames = []
    secs = [int(rate*l/chunk) for l in range(time_s)]
    
    for i in range(int(rate/chunk * time_s)):
        if i in secs:
            print(time_s - secs.index(i))
        data = stream.read(chunk)
        frames.append(data)

    print("finished recording")

    # stop recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    wave_file = wave.open(filename, 'wb')
    wave_file.setnchannels(2)
    wave_file.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
    wave_file.setframerate(rate)
    wave_file.writeframes(b''.join(frames))
    wave_file.close()

In [3]:
def play(filename):
    chunk = 1024

    wf = wave.open(filename, 'rb')

    # instantiate PyAudio
    p = pyaudio.PyAudio()

    # open stream
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)

    # read data
    data = wf.readframes(chunk)

    # play stream
    while len(data) > 0:
        stream.write(data)
        data = wf.readframes(chunk)

    # stop stream
    p.terminate()

In [31]:
def mbari_stream(filename, duration):
    stream_url = 'http://streamingv2.shoutcast.com/ocean-soundscape?lang=en-us'

    r = requests.get(stream_url, stream=True)
    print(f'Recording for {duration}s.\nPress ctrl-C to stop recording.')

    with open(filename, 'wb') as f:
        t0 = time.time()

        for block in r.iter_content(1024):
            if (time.time() - t0) < duration:

                try:
                    f.write(block)

                except KeyboardInterrupt:
                    break
            else:
                break

In [32]:
def plot_spectrogram(filename):
    fs, x = wavfile.read(filename)
    f, t, sxx0 = signal.spectrogram(x[:,0], fs)
    f, t, sxx1 = signal.spectrogram(x[:,1], fs)

    plt.subplot(211)
    plt.pcolormesh(t, f, np.log10(sxx0), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.subplot(212)
    plt.pcolormesh(t, f, np.log10(sxx1), shading='gouraud')
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
    
    return t, f, sxx0, sxx1

In [33]:
filename = 'whale_stream60.wav'

In [34]:
mbari_stream(filename, 15)


Recording for 15s.
Press ctrl-C to stop recording.


In [18]:
record(filename, 60, 44100)

recording...
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
finished recording


In [35]:
plot_spectrogram(filename)


ValueError: File format b'\xff\xfb\x94d' not understood. Only 'RIFF' and 'RIFX' supported.

In [20]:
play(filename)